# Sudoku-9-GraphColoring-Python : Coloration de Graphe (Python)**Navigation** : [<< Human Strategies](Sudoku-8-HumanStrategies-Csharp.ipynb) | [Index](README.md) | [OR-Tools >>](Sudoku-10-ORTools-Python.ipynb)## Objectifs d'apprentissageA la fin de ce notebook, vous saurez :1. **Modéliser** un Sudoku comme un problème de coloration de graphe2. **Comprendre** la théorie des graphes sous-jacente (81 sommets, degré 20)3. **Utiliser** NetworkX pour manipuler des graphes de contraintes4. **Comparer** l'efficacité de différentes stratégies de coloration**Durée estimée** : ~15 min | **Prérequis** : [Sudoku-0 Environment](Sudoku-0-Environment-Csharp.ipynb)---Ce notebook implémente des solveurs de Sudoku utilisant la **coloration de graphe**, une approche classique de la théorie des graphes.

In [ ]:
# Importsimport timefrom typing import List, Optional, Tupleimport networkx as nxprint(f"NetworkX version: {nx.__version__}")print(f"Graphes Sudoku disponibles: {hasattr(nx, 'sudoku_graph')}")

## Introduction : Sudoku et Théorie des GraphesLe Sudoku peut être modélisé comme un **problème de coloration de graphe** :### Modélisation- **Sommets** : 81 cellules de la grille (9x9)- **Arêtes** : deux cellules sont reliées si elles ne peuvent pas avoir la même valeur- **Couleurs** : valeurs 1 à 9### Propriétés du graphe Sudoku- **Nombre de sommets** : 81- **Degré de chaque sommet** : 20- **Nombre d'arêtes** : 810- **Graphe régulier** : tous les sommets ont le même degré

In [ ]:
# Configuration du chemin vers les puzzlesfrom pathlib import PathNOTEBOOK_DIR = Path(r"D:\Dev\CoursIA\MyIA.AI.Notebooks\Sudoku")PUZZLES_DIR = NOTEBOOK_DIR / "Puzzles"def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:    puzzles = []    with open(filepath, "r") as f:        for line in f:            line = line.strip()            if len(line) >= 81:                puzzles.append(line[:81])                if max_puzzles and len(puzzles) >= max_puzzles:                    break    return puzzleseasy_puzzles = load_puzzles(str(PUZZLES_DIR / "Sudoku_Easy51.txt"), max_puzzles=10)print(f"Puzzles charges: {len(easy_puzzles)} faciles")

## Construction du Graphe Sudoku avec NetworkXNetworkX fournit **`nx.sudoku_graph()`** qui génère automatiquement le graphe de contraintes Sudoku !

In [ ]:
# Creer le graphe Sudoku avec NetworkXG = nx.sudoku_graph()print("=== Statistiques du Graphe Sudoku ===")print(f"Sommets: {G.number_of_nodes()}")print(f"Aretes: {G.number_of_edges()}")# Verifier que c'est un graphe regulierdegrees = [d for n, d in G.degree()]print(f"Degre min: {min(degrees)}, max: {max(degrees)}")print(f"Graphe regulier: {len(set(degrees)) == 1}")

### InterpretationNetworkX a genere automatiquement :- **81 sommets** (index 0-80)- **810 aretes** (sans doublons)- **Graphe regulier de degre 20**Chaque sommet represente une cellule, et les aretes representent les contraintes d'exclusion.

## Conversion Grille <-> Graphe

In [ ]:
class SudokuGrid:    """Representation d'une grille de Sudoku 9x9."""    def __init__(self, grid: Optional[List[List[int]]] = None):        if grid is None:            self.cells = [[0] * 9 for _ in range(9)]        else:            self.cells = [row[:] for row in grid]    @classmethod    def from_string(cls, s: str) -> "SudokuGrid":        s = s.replace(".", "0").replace(" ", "").replace("\n", "")        if len(s) != 81:            raise ValueError("La chaine doit avoir 81 caracteres")        grid = cls()        for i in range(81):            grid.cells[i // 9][i % 9] = int(s[i])        return grid    def to_coloring(self) -> List[int]:        """Convertit la grille en coloration de graphe."""        coloring = [0] * 81        for row in range(9):            for col in range(9):                coloring[row * 9 + col] = self.cells[row][col]        return coloring    def from_coloring(self, coloring: List[int]) -> "SudokuGrid":        """Met a jour la grille depuis une coloration."""        for v in range(81):            row, col = v // 9, v % 9            self.cells[row][col] = coloring[v]        return self    def __str__(self) -> str:        lines = []        for r in range(9):            if r > 0 and r % 3 == 0:                lines.append("-" * 21)            row_str = ""            for c in range(9):                if c > 0 and c % 3 == 0:                    row_str += "| "                val = self.cells[r][c]                row_str += (str(val) if val != 0 else ".") + " "            lines.append(row_str)        return "\n".join(lines)# Testtest_grid = SudokuGrid.from_string(easy_puzzles[0])print("Puzzle facile:")print(test_grid)

## Algorithme : Backtracking avec MRVNous utilisons le graphe NetworkX pour guider le backtracking avec l'heuristique MRV (Minimum Remaining Values).

In [ ]:
def solve_with_mrv_backtracking(grid: SudokuGrid) -> Tuple[bool, int, int]:    """    Resout le Sudoku avec backtracking + heuristique MRV.    Returns:        (success, nodes_explored, backtracks)    """    G = nx.sudoku_graph()    coloring = grid.to_coloring()    nodes_explored = 0    backtracks = 0    def get_available_colors(vertex: int, coloring: List[int]) -> set:        used = set()        for neighbor in G.neighbors(vertex):            if coloring[neighbor] != 0:                used.add(coloring[neighbor])        return set(range(1, 10)) - used    def select_mrv_vertex(coloring: List[int]) -> Optional[int]:        best_vertex = None        min_colors = 10        for v in range(81):            if coloring[v] != 0:                continue            available = get_available_colors(v, coloring)            if len(available) < min_colors:                min_colors = len(available)                best_vertex = v        return best_vertex    def backtrack(coloring: List[int]) -> bool:        nonlocal nodes_explored, backtracks        nodes_explored += 1        vertex = select_mrv_vertex(coloring)        if vertex is None:            return True        for color in get_available_colors(vertex, coloring):            coloring[vertex] = color            if backtrack(coloring):                return True            coloring[vertex] = 0            backtracks += 1        return False    success = backtrack(coloring)    grid.from_coloring(coloring)    return success, nodes_explored, backtracks# Testtest_grid = SudokuGrid.from_string(easy_puzzles[0])print("Puzzle a resoudre:")print(test_grid)start = time.time()success, nodes, bts = solve_with_mrv_backtracking(test_grid)elapsed = (time.time() - start) * 1000print(f"\nResolu: {success} en {elapsed:.2f} ms")print(f"Noeuds explores: {nodes}")print(f"Backtracks: {bts}")print("\nSolution:")print(test_grid)

## Benchmark Comparatif

In [ ]:
def benchmark(puzzles: List[str], name: str, limit: int = 5):    """Benchmark de solveur MRV."""    print(f"\nBenchmark: {name} ({min(limit, len(puzzles))} puzzles)")    results = []    for puzzle_str in puzzles[:limit]:        grid = SudokuGrid.from_string(puzzle_str)        start = time.time()        success, nodes, bts = solve_with_mrv_backtracking(grid)        elapsed = (time.time() - start) * 1000        results.append({"success": success, "time_ms": elapsed, "nodes": nodes, "backtracks": bts})    solved = [r for r in results if r["success"]]    if solved:        avg_time = sum(r["time_ms"] for r in solved) / len(solved)        print(f"Succes: {len(solved)}/{len(results)}")        print(f"Temps moyen: {avg_time:.2f} ms")    else:        print("Aucun puzzle resolu !")    return resultsbenchmark(easy_puzzles, "Puzzles Faciles", limit=3)hard_puzzles = load_puzzles(str(PUZZLES_DIR / "Sudoku_hardest.txt"))benchmark(hard_puzzles, "Puzzles Difficiles", limit=2)

## Resume### NetworkX pour Sudoku| Avantages | Inconvenients ||-----------|---------------|| `nx.sudoku_graph()` pret a l'emploi | Pas de solveur complet integre || Algorithmes de coloration varies | Moins performant que CP-SAT || Facile d'experimentation | Necessite backtracking manuel |### Au-dela du SudokuNetworkX peut resoudre de nombreux problemes de coloration :- Cartes geographiques- Emploi du temps- Allocation de frequences---**Navigation** : [<< Human Strategies](Sudoku-8-HumanStrategies-Csharp.ipynb) | [Index](README.md) | [OR-Tools >>](Sudoku-10-ORTools-Python.ipynb)